In [ ]:
!pip install --q ultralytics supervision torch torchvision transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 20.9 MB/s eta 0:00:00


In [ ]:
%%capture
!gdown --q --folder 1n98ur5MdsKuRtXTHkeM4PEwklBkDaJe4
!unzip --q /content/Data/observing.zip
!unzip --q /content/Data/public_test.zip

Archive:  /content/Data/observing.zip
   creating: train/
   creating: train/samples/
   creating: train/samples/Backpack_0/
   creating: train/samples/Backpack_0/object_images/
  inflating: train/samples/Backpack_0/object_images/img_1.jpg  
  inflating: train/samples/Backpack_0/object_images/img_2.jpg  
  inflating: train/samples/Backpack_0/object_images/img_3.jpg  
  inflating: train/samples/Backpack_0/drone_video.mp4  
   creating: train/samples/Backpack_1/
   creating: train/samples/Backpack_1/object_images/
  inflating: train/samples/Backpack_1/object_images/img_1.jpg  
  inflating: train/samples/Backpack_1/object_images/img_2.jpg  
  inflating: train/samples/Backpack_1/object_images/img_3.jpg  
  inflating: train/samples/Backpack_1/drone_video.mp4  
   creating: train/samples/Jacket_0/
   creating: train/samples/Jacket_0/object_images/
  inflating: train/samples/Jacket_0/object_images/img_1.jpg  
  inflating: train/samples/Jacket_0/object_images/img_2.jpg  
  inflating: train/sam

In [ ]:
# -*- coding: utf-8 -*-
"""
COMPLETE OPTIMIZED PIPELINE: YOLO World + Masked Frame Training (Mixed Curriculum)
Tối ưu để giảm số tham số xuống dưới 50M
"""

# ============================================
# IMPORTS
# ============================================

import os
import json
import cv2
import yaml
import random
import numpy as np
import torch
import torch.nn as nn
from scipy.interpolate import interp1d
from concurrent.futures import ThreadPoolExecutor, as_completed
from ultralytics import YOLOWorld

# ============================================
# CONFIGURATION (ĐÃ TỐI ƯU)
# ============================================

class Config:
    """Cấu hình cho pipeline training"""
    # Dataset paths
    DATASET_ROOT = "train"
    ANNOTATIONS_PATH = os.path.join(DATASET_ROOT, "annotations/annotations.json")
    SAMPLES_DIR = os.path.join(DATASET_ROOT, "samples")
    WORK_DIR = "enhanced_mixed_dataset"

    # Training settings
    TRAIN_RATIO = 0.8
    IMG_EXT = "jpg"
    FRAME_STEP = 1
    NUM_WORKERS = 8

    # ========================================
    # 🔥 THAY ĐỔI QUAN TRỌNG: Model nhỏ hơn
    # ========================================
    MODEL_WEIGHTS = "yolov8s-worldv2.pt"  # ✅ ~11M params base (thay vì yolov8m 28M)
    # Nếu muốn cực nhỏ: "yolov8n-worldv2.pt"  # ✅ ~3M params base

    CLASS_NAMES = ["target"]

    # Parameter limit
    PARAM_LIMIT = 50_000_000  # 50M params limit

    # Masked training settings
    ENABLE_MASKING = True
    NUM_AUGMENTATIONS_PER_PHASE = 5

    # Curriculum Learning settings
    CURRICULUM = {
        'phase1': {'epochs': (0, 15), 'mask_ratio': 0.1, 'strategy': 'random'},
        'phase2': {'epochs': (15, 35), 'mask_ratio': 0.3, 'strategy': 'span'},
        'phase3': {'epochs': (35, 50), 'mask_ratio': 0.5, 'strategy': 'keyframe'}
    }

config = Config()
print("✅ Configuration loaded")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Configuration loaded


In [ ]:
# ============================================
# MODEL OPTIMIZER CLASS
# ============================================

class ModelOptimizer:
    """Tối ưu model YOLO World"""

    def __init__(self, model):
        self.model = model

    def count_parameters(self):
        """Đếm số tham số"""
        model = self.model.model if hasattr(self.model, 'model') else self.model

        total = sum(p.numel() for p in model.parameters())
        trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

        return {
            'total': total,
            'trainable': trainable,
            'non_trainable': total - trainable
        }

    def print_model_summary(self, stage=""):
        """In thống kê model"""
        params = self.count_parameters()

        print(f"\n{'='*60}")
        print(f"📊 MODEL STATISTICS {stage}")
        print(f"{'='*60}")
        print(f"   Total params: {params['total']:,}")
        print(f"   Trainable: {params['trainable']:,}")
        print(f"   Non-trainable: {params['non_trainable']:,}")
        print(f"   Memory (FP32): {params['total'] * 4 / 1024**2:.2f} MB")
        print(f"   Memory (FP16): {params['total'] * 2 / 1024**2:.2f} MB")

        # Check limit
        if params['total'] > config.PARAM_LIMIT:
            print(f"\n   ❌ EXCEEDED LIMIT: {params['total'] - config.PARAM_LIMIT:,} params over")
            print(f"   Limit: {config.PARAM_LIMIT:,}")
        else:
            print(f"\n   ✅ WITHIN LIMIT: {config.PARAM_LIMIT - params['total']:,} params remaining")

        print(f"{'='*60}\n")

        return params

    def apply_magnitude_pruning(self, prune_ratio=0.3):
        """
        Magnitude-based pruning: Xóa weights có giá trị tuyệt đối nhỏ
        """
        print(f"\n🔧 Applying {prune_ratio*100:.0f}% Magnitude Pruning...")

        model = self.model.model if hasattr(self.model, 'model') else self.model

        # Thu thập tất cả weights
        all_weights = []
        for name, module in model.named_modules():
            if isinstance(module, (nn.Conv2d, nn.Linear)):
                if hasattr(module, 'weight') and module.weight is not None:
                    all_weights.append(module.weight.data.abs().flatten())

        if not all_weights:
            print("⚠️ No weights found for pruning!")
            return

        # Tính threshold
        all_weights_tensor = torch.cat(all_weights)
        threshold = torch.quantile(all_weights_tensor, prune_ratio)

        print(f"   Threshold: {threshold:.6f}")

        # Áp dụng pruning
        pruned_count = 0
        total_count = 0

        for name, module in model.named_modules():
            if isinstance(module, (nn.Conv2d, nn.Linear)):
                if hasattr(module, 'weight') and module.weight is not None:
                    weight = module.weight.data
                    mask = weight.abs() > threshold
                    weight.mul_(mask)

                    pruned_count += (~mask).sum().item()
                    total_count += weight.numel()

        print(f"   ✅ Pruned {pruned_count:,} / {total_count:,} weights "
              f"({pruned_count/total_count*100:.2f}%)")

        return self.model

print("✅ Model optimizer class loaded")


✅ Model optimizer class loaded


In [ ]:
# ============================================
# 1. MASKING STRATEGIES
# ============================================

class FrameMaskingStrategy:
    """Các chiến lược che frames để tạo self-supervised learning"""

    @staticmethod
    def random_mask(total_frames, mask_ratio=0.3):
        """Che ngẫu nhiên mask_ratio% frames"""
        if total_frames < 3:
            return []
        num_mask = max(1, int(total_frames * mask_ratio))
        num_mask = min(num_mask, total_frames - 2)  # Keep at least 2 visible
        if num_mask <= 0:
            return []
        try:
            masked_indices = random.sample(range(1, total_frames-1), num_mask)
            return sorted(masked_indices)
        except ValueError:
            # Nếu không đủ frames để sample
            return []

    @staticmethod
    def span_mask(total_frames, mask_ratio=0.3):
        """Che các đoạn liên tiếp"""
        if total_frames < 5:
            return FrameMaskingStrategy.random_mask(total_frames, mask_ratio)

        span_length = max(2, int(total_frames * 0.1))
        num_spans = max(1, int(total_frames * mask_ratio / span_length))

        masked_indices = []
        for _ in range(num_spans):
            if total_frames - span_length - 1 <= 1:
                start = 1
            else:
                start = random.randint(1, max(2, total_frames - span_length - 1))
            masked_indices.extend(range(start, min(start + span_length, total_frames-1)))

        return sorted(list(set(masked_indices)))

    @staticmethod
    def keyframe_mask(frame_boxes, mask_ratio=0.3):
        """Ưu tiên mask frames có chuyển động lớn"""
        if len(frame_boxes) < 3:
            return []

        # Calculate motion scores
        motion_scores = []
        frame_indices = sorted(frame_boxes.keys())

        for i in range(1, len(frame_indices) - 1):
            prev_frame = frame_indices[i-1]
            curr_frame = frame_indices[i]

            if not frame_boxes.get(prev_frame) or not frame_boxes.get(curr_frame):
                continue

            try:
                prev_box = frame_boxes[prev_frame][0]
                curr_box = frame_boxes[curr_frame][0]

                # Calculate center displacement
                prev_cx = (prev_box['x1'] + prev_box['x2']) / 2
                prev_cy = (prev_box['y1'] + prev_box['y2']) / 2
                curr_cx = (curr_box['x1'] + curr_box['x2']) / 2
                curr_cy = (curr_box['y1'] + curr_box['y2']) / 2

                motion = np.sqrt((curr_cx - prev_cx)**2 + (curr_cy - prev_cy)**2)
                motion_scores.append((curr_frame, motion))
            except (IndexError, KeyError):
                continue

        if not motion_scores:
            return FrameMaskingStrategy.random_mask(len(frame_indices), mask_ratio)

        # Sort by motion and mask top frames
        motion_scores.sort(key=lambda x: x[1], reverse=True)
        num_mask = max(1, int(len(motion_scores) * mask_ratio))
        masked_frames = [frame for frame, _ in motion_scores[:num_mask]]

        return sorted(masked_frames)

    @staticmethod
    def block_mask(total_frames, mask_ratio=0.3):
        """Che theo blocks"""
        if total_frames < 5:
            return FrameMaskingStrategy.random_mask(total_frames, mask_ratio)

        block_size = max(3, int(total_frames * 0.05))
        num_blocks = total_frames // block_size
        if num_blocks < 3:
            return FrameMaskingStrategy.random_mask(total_frames, mask_ratio)

        num_mask_blocks = max(1, int(num_blocks * mask_ratio))

        try:
            masked_blocks = random.sample(range(1, num_blocks-1),
                                          min(num_mask_blocks, num_blocks-2))
        except ValueError:
            return []

        masked_indices = []
        for block_idx in masked_blocks:
            start = block_idx * block_size
            end = min(start + block_size, total_frames - 1)
            masked_indices.extend(range(start, end))

        return sorted(masked_indices)

print("✅ Masking strategies loaded")


✅ Masking strategies loaded


In [ ]:
# ============================================
# 2. TEMPORAL INTERPOLATION
# ============================================

def interpolate_boxes(frame_boxes, masked_frames, method='cubic'):
    """
    Nội suy bounding boxes cho frames bị mask

    Args:
        frame_boxes: Dict mapping frame_idx -> list of bbox dicts
        masked_frames: List of frame indices to interpolate
        method: Interpolation method ('linear', 'cubic', etc.)

    Returns:
        ground_truth: Dict of interpolated boxes for masked frames
        visible_frames: List of visible frame indices
    """
    all_frame_indices = sorted(frame_boxes.keys())
    visible_frames = [f for f in all_frame_indices if f not in masked_frames]

    if len(visible_frames) < 2:
        return {}, all_frame_indices  # Cannot interpolate

    ground_truth = {}

    # Extract coordinates from visible frames (assume single object)
    visible_data = [(f, frame_boxes[f][0]) for f in visible_frames
                    if frame_boxes[f] and len(frame_boxes[f]) > 0]

    if len(visible_data) < 2:
        return {}, visible_frames

    try:
        frames = [d[0] for d in visible_data]
        x1_vals = [d[1]['x1'] for d in visible_data]
        y1_vals = [d[1]['y1'] for d in visible_data]
        x2_vals = [d[1]['x2'] for d in visible_data]
        y2_vals = [d[1]['y2'] for d in visible_data]

        # Create interpolation functions
        kind = 'linear' if len(frames) == 2 else method
        f_x1 = interp1d(frames, x1_vals, kind=kind, bounds_error=False,
                        fill_value=(x1_vals[0], x1_vals[-1]))
        f_y1 = interp1d(frames, y1_vals, kind=kind, bounds_error=False,
                        fill_value=(y1_vals[0], y1_vals[-1]))
        f_x2 = interp1d(frames, x2_vals, kind=kind, bounds_error=False,
                        fill_value=(x2_vals[0], x2_vals[-1]))
        f_y2 = interp1d(frames, y2_vals, kind=kind, bounds_error=False,
                        fill_value=(y2_vals[0], y2_vals[-1]))

        # Generate ground truth for masked frames
        for frame_idx in masked_frames:
            if frame_idx in frame_boxes:  # Only interpolate existing frames
                ground_truth[frame_idx] = [{
                    'frame': frame_idx,
                    'x1': int(np.clip(f_x1(frame_idx), 0, 10000)),
                    'y1': int(np.clip(f_y1(frame_idx), 0, 10000)),
                    'x2': int(np.clip(f_x2(frame_idx), 0, 10000)),
                    'y2': int(np.clip(f_y2(frame_idx), 0, 10000))
                }]
    except Exception as e:
        print(f"⚠️ Interpolation failed: {e}")
        return {}, visible_frames

    return ground_truth, visible_frames

print("✅ Temporal interpolation function loaded")


✅ Temporal interpolation function loaded


In [ ]:
# ============================================
# 3. REMOVE DUPLICATE BOXES
# ============================================

def remove_duplicate_boxes(boxes, iou_threshold=0.95):
    """
    Loại bỏ các boxes trùng lặp dựa trên IoU threshold

    Args:
        boxes: List of bbox dicts with keys ['x1', 'y1', 'x2', 'y2']
        iou_threshold: IoU threshold để xác định trùng lặp

    Returns:
        Filtered list of boxes
    """
    if len(boxes) <= 1:
        return boxes

    def calculate_iou(box1, box2):
        """Tính IoU giữa 2 boxes"""
        x1_min, y1_min, x1_max, y1_max = box1
        x2_min, y2_min, x2_max, y2_max = box2

        inter_x_min = max(x1_min, x2_min)
        inter_y_min = max(y1_min, y2_min)
        inter_x_max = min(x1_max, x2_max)
        inter_y_max = min(y1_max, y2_max)

        if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
            return 0.0

        inter_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
        box1_area = (x1_max - x1_min) * (y1_max - y1_min)
        box2_area = (x2_max - x2_min) * (y2_max - y2_min)

        union_area = box1_area + box2_area - inter_area
        return inter_area / union_area if union_area > 0 else 0.0

    boxes_xyxy = [(bb['x1'], bb['y1'], bb['x2'], bb['y2']) for bb in boxes]
    keep = [True] * len(boxes)

    for i in range(len(boxes)):
        if not keep[i]:
            continue
        for j in range(i + 1, len(boxes)):
            if not keep[j]:
                continue

            iou = calculate_iou(boxes_xyxy[i], boxes_xyxy[j])
            if iou > iou_threshold:
                area_i = (boxes[i]['x2'] - boxes[i]['x1']) * (boxes[i]['y2'] - boxes[i]['y1'])
                area_j = (boxes[j]['x2'] - boxes[j]['x1']) * (boxes[j]['y2'] - boxes[j]['y1'])

                if area_i >= area_j:
                    keep[j] = False
                else:
                    keep[i] = False
                    break

    return [boxes[i] for i in range(len(boxes)) if keep[i]]

print("✅ Duplicate removal function loaded")


✅ Duplicate removal function loaded


In [ ]:
# ============================================
# 4. ENHANCED FRAME EXTRACTION (Với Masking)
# ============================================

def extract_frames_with_masking(
    video_id,
    ann_dict,
    mode="train",
    augmentation_id=0,
    mask_strategy='random',
    mask_ratio=0.3
):
    """
    Extract frames + labels với masked frame augmentation

    Args:
        video_id: ID của video
        ann_dict: Dictionary chứa annotations
        mode: "train" hoặc "val"
        augmentation_id: ID duy nhất cho augmentation
        mask_strategy: Chiến lược mask ('random', 'span', 'keyframe', 'block')
        mask_ratio: Tỷ lệ frames bị mask

    Returns:
        Dict với status và thống kê
    """
    video_dir = os.path.join(config.SAMPLES_DIR, video_id)
    video_path = os.path.join(video_dir, "drone_video.mp4")

    if not os.path.exists(video_path):
        return {'status': 'missing_video', 'video_id': video_id}

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        cap.release()
        return {'status': 'cannot_open', 'video_id': video_id}

    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Build bbox_dict với deduplication
    bbox_dict = {}
    dup_removed = 0

    for interval in ann_dict.get(video_id, {}).get("annotations", []):
        for bb in interval.get("bboxes", []):
            frame_idx = bb["frame"]
            bbox_dict.setdefault(frame_idx, []).append(bb)

    # Remove duplicates
    for frame_idx in bbox_dict:
        original_count = len(bbox_dict[frame_idx])
        bbox_dict[frame_idx] = remove_duplicate_boxes(bbox_dict[frame_idx], iou_threshold=0.95)
        dup_removed += original_count - len(bbox_dict[frame_idx])

    # Apply masking strategy (if enabled and ratio > 0)
    masked_frames = []
    ground_truth = {}

    if config.ENABLE_MASKING and mask_ratio > 0 and len(bbox_dict) > 4:
        masker = FrameMaskingStrategy()

        # Lấy danh sách các frame_indices có label
        frame_indices = sorted(bbox_dict.keys())
        num_frames_with_labels = len(frame_indices)

        if mask_strategy == 'random':
            masked_idx_list = masker.random_mask(num_frames_with_labels, mask_ratio)
        elif mask_strategy == 'span':
            masked_idx_list = masker.span_mask(num_frames_with_labels, mask_ratio)
        elif mask_strategy == 'keyframe':
            # keyframe_mask tự trả về frame_idx, không phải index
            masked_frames = masker.keyframe_mask(bbox_dict, mask_ratio)
            masked_idx_list = []  # Đánh dấu là đã xử lý
        elif mask_strategy == 'block':
            masked_idx_list = masker.block_mask(num_frames_with_labels, mask_ratio)
        else:
            masked_idx_list = []

        # Chuyển đổi từ list index (0, 1, 2...) sang frame index (120, 125, 130...)
        if masked_idx_list:
            masked_frames = [frame_indices[i] for i in masked_idx_list
                             if i < len(frame_indices)]

        # Generate ground truth for masked frames
        if masked_frames:
            ground_truth, _ = interpolate_boxes(bbox_dict, masked_frames, method='cubic')

    # Output directories
    if mode == "train":
        img_out = os.path.join(config.WORK_DIR, 'train', 'images')
        lbl_out = os.path.join(config.WORK_DIR, 'train', 'labels')
    else:
        img_out = os.path.join(config.WORK_DIR, 'val', 'images')
        lbl_out = os.path.join(config.WORK_DIR, 'val', 'labels')

    saved = 0
    masked_count = 0
    idx = 0

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Chỉ lưu các frame có trong annotation
            if idx % config.FRAME_STEP == 0 and idx in bbox_dict:
                # Filename với augmentation ID DUY NHẤT
                img_name = f"{video_id}_aug{augmentation_id:04d}_frame_{idx:06d}.{config.IMG_EXT}"
                img_path = os.path.join(img_out, img_name)

                txt_name = f"{video_id}_aug{augmentation_id:04d}_frame_{idx:06d}.txt"
                txt_path = os.path.join(lbl_out, txt_name)

                # Use ground truth for masked frames, original for visible
                if idx in masked_frames and idx in ground_truth:
                    boxes_to_save = ground_truth[idx]
                    masked_count += 1
                elif idx in bbox_dict:
                    boxes_to_save = bbox_dict[idx]
                else:
                    boxes_to_save = []  # Không nên xảy ra nếu logic đúng

                # Chỉ lưu ảnh nếu frame đó được chọn
                if boxes_to_save:
                    if not cv2.imwrite(img_path, frame):
                        continue  # Skip if write failed

                    lines = []
                    for bb in boxes_to_save:
                        x1, y1, x2, y2 = bb["x1"], bb["y1"], bb["x2"], bb["y2"]

                        # Validate bbox
                        if x2 <= x1 or y2 <= y1:
                            continue
                        if x1 < 0 or y1 < 0 or x2 > w or y2 > h:
                            x1, y1 = max(0, x1), max(0, y1)
                            x2, y2 = min(w, x2), min(h, y2)

                        cx, cy = (x1 + x2) / 2 / w, (y1 + y2) / 2 / h
                        bw, bh = (x2 - x1) / w, (y2 - y1) / h

                        if not (0 <= cx <= 1 and 0 <= cy <= 1):
                            continue
                        if not (0 < bw <= 1 and 0 < bh <= 1):
                            continue

                        lines.append(f"0 {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

                    with open(txt_path, "w") as f:
                        f.write("\n".join(lines))

                    saved += 1

            idx += 1
    finally:
        cap.release()

    return {
        'status': 'success',
        'video_id': video_id,
        'aug_id': augmentation_id,
        'frames_saved': saved,
        'masked_frames': masked_count,
        'duplicates_removed': dup_removed
    }

print("✅ Frame extraction function loaded")


✅ Frame extraction function loaded


In [ ]:
# ============================================
# 5. CURRICULUM LEARNING CONTROLLER
# ============================================

class CurriculumController:
    """Điều khiển curriculum learning qua các epochs (chỉ để giám sát)"""

    def __init__(self, curriculum_config):
        self.curriculum = curriculum_config
        self.phases = sorted(curriculum_config.items(),
                            key=lambda x: x[1]['epochs'][0])

    def get_phase(self, epoch):
        """Lấy phase hiện tại dựa trên epoch"""
        for phase_name, phase_config in self.phases:
            start_epoch, end_epoch = phase_config['epochs']
            if start_epoch <= epoch < end_epoch:
                return phase_name, phase_config

        # Return last phase if beyond
        if self.phases:
            return self.phases[-1][0], self.phases[-1][1]
        return None, None

    def get_config(self, epoch):
        """Lấy config cho epoch hiện tại"""
        phase_name, phase_config = self.get_phase(epoch)
        if phase_config is None:
            return {'phase': 'unknown', 'mask_ratio': 0.0, 'strategy': 'random'}
        return {
            'phase': phase_name,
            'mask_ratio': phase_config['mask_ratio'],
            'strategy': phase_config['strategy']
        }

    def print_schedule(self):
        """In lịch trình curriculum"""
        print("\n" + "="*60)
        print("📚 CURRICULUM LEARNING SCHEDULE (FOR DATA GENERATION)")
        print("="*60)
        for phase_name, phase_config in self.phases:
            start, end = phase_config['epochs']
            print(f"\n{phase_name.upper()}: (Reference for Epochs {start}-{end})")
            print(f"  Mask Ratio: {phase_config['mask_ratio']*100:.0f}%")
            print(f"  Strategy: {phase_config['strategy']}")
            print(f"  Augmentations: {config.NUM_AUGMENTATIONS_PER_PHASE} versions")
        print("="*60 + "\n")

print("✅ Curriculum controller loaded")


✅ Curriculum controller loaded


In [ ]:
# ============================================
# MODEL ANALYSIS UTILITY
# ============================================

def analyze_model(model_path):
    """Phân tích chi tiết model"""

    print("\n" + "="*60)
    print("🔍 MODEL ANALYSIS")
    print("="*60)

    model = YOLOWorld(model_path)
    model.set_classes(config.CLASS_NAMES)

    optimizer = ModelOptimizer(model)
    params = optimizer.count_parameters()

    # Detailed layer analysis
    print("\n📊 Layer-wise Parameter Count:")
    print("-" * 60)

    layer_params = {}
    for name, module in model.model.named_modules():
        if len(list(module.children())) == 0:  # Leaf modules only
            num_params = sum(p.numel() for p in module.parameters())
            if num_params > 0:
                layer_type = type(module).__name__
                if layer_type not in layer_params:
                    layer_params[layer_type] = {'count': 0, 'params': 0}
                layer_params[layer_type]['count'] += 1
                layer_params[layer_type]['params'] += num_params

    # Sort by parameter count
    sorted_layers = sorted(layer_params.items(), key=lambda x: x[1]['params'], reverse=True)

    for layer_type, info in sorted_layers[:10]:  # Top 10
        print(f"   {layer_type:20s}: {info['count']:3d} layers, {info['params']:12,d} params "
              f"({info['params']/params['total']*100:5.2f}%)")

    print("-" * 60)
    print(f"   {'TOTAL':20s}: {params['total']:,} params\n")

    # Memory estimation
    print("💾 Memory Estimates:")
    print(f"   FP32 (training): {params['total'] * 4 / 1024**2:.2f} MB")
    print(f"   FP16 (inference): {params['total'] * 2 / 1024**2:.2f} MB")
    print(f"   INT8 (optimized): {params['total'] * 1 / 1024**2:.2f} MB")

    # Check limit
    print(f"\n⚖️  Parameter Limit Check:")
    if params['total'] <= config.PARAM_LIMIT:
        print(f"   ✅ Within limit: {params['total']:,} / {config.PARAM_LIMIT:,}")
        print(f"   Remaining: {config.PARAM_LIMIT - params['total']:,} params")
    else:
        print(f"   ❌ Exceeds limit: {params['total']:,} / {config.PARAM_LIMIT:,}")
        print(f"   Over by: {params['total'] - config.PARAM_LIMIT:,} params")
        print(f"\n   Recommended actions:")
        print(f"   1. Apply pruning (30-40% reduction)")
        print(f"   2. Use smaller base model (yolov8n)")
        print(f"   3. Reduce model depth/width")

    print("="*60)

print("✅ Model analysis function loaded")


✅ Model analysis function loaded


In [ ]:
# ============================================
# 6. MAIN PIPELINE
# ============================================

def main_pipeline():
    """Main training pipeline với curriculum learning (Mixed Dataset)"""

    print("🚀 ENHANCED YOLO WORLD TRAINING PIPELINE (MIXED DATA)")
    print("="*60)

    # Load annotations
    with open(config.ANNOTATIONS_PATH, "r") as f:
        annotations = json.load(f)

    video_ids = [a["video_id"] for a in annotations]
    ann_dict = {a["video_id"]: a for a in annotations}

    # Split train/val
    random.seed(42)
    random.shuffle(video_ids)
    split_idx = int(len(video_ids) * config.TRAIN_RATIO)
    train_videos = video_ids[:split_idx]
    val_videos = video_ids[split_idx:]

    print(f"✅ Loaded {len(video_ids)} videos")
    print(f"    Train: {len(train_videos)}, Val: {len(val_videos)}")

    # Setup curriculum
    curriculum = CurriculumController(config.CURRICULUM)
    curriculum.print_schedule()

    # Create output directories
    os.makedirs(config.WORK_DIR, exist_ok=True)
    for subdir in ['train/images', 'train/labels', 'val/images', 'val/labels']:
        os.makedirs(os.path.join(config.WORK_DIR, subdir), exist_ok=True)

    # Tạo một danh sách job duy nhất chứa TẤT CẢ các augmentation
    print(f"\n⚙️ Preparing all data augmentations in one go...")
    futures = []
    stats_list = []
    global_aug_id_counter = 0  # ID tăng cường duy nhất

    with ThreadPoolExecutor(max_workers=config.NUM_WORKERS) as ex:

        # 1. Xử lý TRAIN data với TẤT CẢ các phase curriculum
        for phase_name, phase_config in curriculum.phases:
            print(f"  -> Submitting jobs for Phase: {phase_name.upper()} "
                  f"({config.NUM_AUGMENTATIONS_PER_PHASE} versions each)")

            mask_ratio = phase_config['mask_ratio']
            mask_strategy = phase_config['strategy']

            for vid in train_videos:
                for _ in range(config.NUM_AUGMENTATIONS_PER_PHASE):
                    futures.append(
                        ex.submit(
                            extract_frames_with_masking,
                            vid, ann_dict, "train",
                            global_aug_id_counter,
                            mask_strategy, mask_ratio
                        )
                    )
                    global_aug_id_counter += 1

        print(f"\n  -> Submitting jobs for VALIDATION data (1 version, no masking)")
        # 2. Xử lý VAL data (chỉ 1 lần, không mask, aug_id = 0)
        for vid in val_videos:
            futures.append(
                ex.submit(
                    extract_frames_with_masking,
                    vid, ann_dict, "val",
                    0,  # aug_id 0 cho val
                    'random', 0.0  # mask_ratio = 0.0
                )
            )

        print(f"\n⏳ Waiting for {len(futures)} total jobs to complete...")
        # 3. Thu thập kết quả
        for i, fut in enumerate(as_completed(futures), 1):
            result = fut.result()
            stats_list.append(result)

            if i % 100 == 0 or i == len(futures):
                if result['status'] == 'success':
                    msg = (f"[{i}/{len(futures)}] ✅ {result['video_id']}_aug{result['aug_id']:04d}: "
                           f"{result['frames_saved']} frames")
                    if result['masked_frames'] > 0:
                        msg += f" ({result['masked_frames']} masked)"
                    print(msg)
                else:
                    print(f"[{i}/{len(futures)}] ⚠️ {result['video_id']}: {result['status']}")

    # In thống kê tổng
    success_stats = [s for s in stats_list if s['status'] == 'success']
    total_frames = sum(s['frames_saved'] for s in success_stats)
    total_masked = sum(s['masked_frames'] for s in success_stats)

    print(f"\n{'='*60}")
    print(f"📈 TOTAL DATASET STATISTICS (ALL PHASES MIXED):")
    print(f"   Total frames saved: {total_frames}")
    print(f"   Total masked frames: {total_masked} ({total_masked/max(1,total_frames)*100:.1f}%)")
    print(f"   Total train augmentations: {global_aug_id_counter}")
    print(f"{'='*60}")

    # Create data.yaml
    data_yaml = {
        "train": os.path.abspath(os.path.join(config.WORK_DIR, 'train', 'images')),
        "val": os.path.abspath(os.path.join(config.WORK_DIR, 'val', 'images')),
        "nc": 1,
        "names": config.CLASS_NAMES
    }
    data_path = os.path.join(config.WORK_DIR, "data.yaml")
    with open(data_path, "w") as f:
        yaml.dump(data_yaml, f)

    print(f"\n📄 data.yaml created at: {data_path}")
    print(open(data_path).read())

    # Training
    print("\n" + "="*60)
    print("🚀 STARTING TRAINING ON MIXED DATASET")
    print("="*60)

    model = YOLOWorld(config.MODEL_WEIGHTS)
    model.set_classes(config.CLASS_NAMES)

    # Custom training callback (chỉ để giám sát)
    monitor = CurriculumController(config.CURRICULUM)

    def on_epoch_end(trainer):
        """Callback để IN RA phase tương ứng với epoch"""
        epoch = trainer.epoch + 1  # trainer.epoch là 0-indexed
        phase_config = monitor.get_config(epoch)

        if epoch == 1 or (epoch) % 5 == 0 or epoch == trainer.epochs:
            print(f"\n-- 📚 Epoch {epoch}/{trainer.epochs} -- "
                  f"Curriculum Phase (Reference): {phase_config['phase']} --")

    # Thêm callback TRƯỚC KHI train
    model.add_callback("on_epoch_end", on_epoch_end)

    # Train
    results = model.train(
        data=data_path,
        epochs=50,
        imgsz=896,
        batch=32,
        lr0=5e-4,
        optimizer="AdamW",
        box=10.0,
        cls=0.5,
        dfl=1.5,

        # Data augmentation (temporal-friendly)
        mosaic=0.3,  # Giảm mosaic
        mixup=0.0,
        copy_paste=0.0,
        degrees=10.0,
        translate=0.1,
        scale=0.5,
        flipud=0.0,
        fliplr=0.5,

        # Regularization
        dropout=0.0,
        weight_decay=0.0005,

        # Training settings
        patience=10,
        save_period=5,
        workers=8,
        close_mosaic=10,

        # Output
        project=os.path.join(config.WORK_DIR, "runs"),
        name="curriculum_MIXED_training",
        exist_ok=True,
        pretrained=True,
        verbose=True
    )

    print("\n" + "="*60)
    print("✅ TRAINING COMPLETE!")
    print("="*60)
    best_model_path = os.path.join(model.trainer.save_dir, 'weights', 'best.pt')
    print(f"Best model: {best_model_path}")

    return best_model_path

print("✅ Main pipeline function loaded")


✅ Main pipeline function loaded


In [ ]:
# ============================================
# 7. INFERENCE
# ============================================

def run_inference(model_path, test_root, output_json, conf=0.35, iou=0.55):
    """Run inference trên test set"""

    print("\n" + "="*60)
    print("🎯 RUNNING INFERENCE")
    print("="*60)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = YOLOWorld(model_path)
    model.set_classes(config.CLASS_NAMES)

    video_dirs = sorted([d for d in os.listdir(test_root)
                         if os.path.isdir(os.path.join(test_root, d))])
    submission = []

    for i, vid in enumerate(video_dirs, 1):
        video_path = os.path.join(test_root, vid, "drone_video.mp4")
        if not os.path.exists(video_path):
            continue

        bboxes_per_frame = []

        results = model.predict(
            source=video_path,
            conf=conf,
            iou=iou,
            imgsz=896,
            stream=True,
            verbose=False,
            device=device
        )

        frame_idx = 0
        for r in results:
            if r.boxes is not None and len(r.boxes) > 0:
                boxes = r.boxes.xyxy.cpu().numpy()
                for (x1, y1, x2, y2) in boxes:
                    bboxes_per_frame.append({
                        "frame": frame_idx,
                        "x1": int(x1),
                        "y1": int(y1),
                        "x2": int(x2),
                        "y2": int(y2)
                    })
            frame_idx += 1

        submission.append({
            "video_id": vid,
            "detections": [{"bboxes": bboxes_per_frame}] if bboxes_per_frame else []
        })

        print(f"[{i}/{len(video_dirs)}] {vid}: {len(bboxes_per_frame)} detections")

    with open(output_json, "w") as f:
        json.dump(submission, f, indent=2, ensure_ascii=False)

    print(f"\n✅ Saved submission to: {output_json}")

print("✅ Inference function loaded")


✅ Inference function loaded


In [ ]:
# ============================================
# 8. MAIN EXECUTION
# ============================================

print("""
╔═══════════════════════════════════════════════════════════╗
║    YOLO WORLD + MASKED TRAINING + CURRICULUM (MIXED)    ║
║    Enhanced Pipeline for Video Object Detection         ║
╚═══════════════════════════════════════════════════════════╝
""")

# Check GPU
print(f"🖥️  Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    print(f"    GPU: {torch.cuda.get_device_name(0)}")

# Cấu hình đường dẫn
config.DATASET_ROOT = "train"  # Thay đổi nếu cần
TEST_ROOT = "public_test/samples"  # Thay đổi nếu cần
OUTPUT_JSON = "submission_mixed.json"

# Run main pipeline
try:
    best_model_path = main_pipeline()

    # Optional: Run inference
    print("\n" + "="*60)

    if os.path.exists(TEST_ROOT) and best_model_path:
        print("Auto-running inference on test set...")
        run_inference(
            model_path=best_model_path,
            test_root=TEST_ROOT,
            output_json=OUTPUT_JSON
        )
    else:
        print("Skipping inference: Test set path not found or training failed.")

except Exception as e:
    print(f"\n❌ An error occurred in the pipeline: {e}")
    import traceback
    traceback.print_exc()



╔═══════════════════════════════════════════════════════════╗
║    YOLO WORLD + MASKED TRAINING + CURRICULUM (MIXED)    ║
║    Enhanced Pipeline for Video Object Detection         ║
╚═══════════════════════════════════════════════════════════╝

🖥️  Device: CUDA
    GPU: NVIDIA A100-SXM4-40GB
🚀 ENHANCED YOLO WORLD TRAINING PIPELINE (MIXED DATA)
✅ Loaded 14 videos
    Train: 11, Val: 3

📚 CURRICULUM LEARNING SCHEDULE (FOR DATA GENERATION)

PHASE1: (Reference for Epochs 0-15)
  Mask Ratio: 10%
  Strategy: random
  Augmentations: 5 versions

PHASE2: (Reference for Epochs 15-35)
  Mask Ratio: 30%
  Strategy: span
  Augmentations: 5 versions

PHASE3: (Reference for Epochs 35-50)
  Mask Ratio: 50%
  Strategy: keyframe
  Augmentations: 5 versions


⚙️ Preparing all data augmentations in one go...
  -> Submitting jobs for Phase: PHASE1 (5 versions each)
  -> Submitting jobs for Phase: PHASE2 (5 versions each)
  -> Submitting jobs for Phase: PHASE3 (5 versions each)

  -> Submitting jobs for VA

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 123MiB/s]


Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=10.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=enhanced_mixed_dataset/data.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-worldv2.pt, momentum=0.937, mosaic=0.3, multi_scale=False, name=curriculum_MIXED_training, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, 

Exception in thread Thread-16 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 61, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 37, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/resource_

KeyboardInterrupt: 